In [1]:
from pathlib import Path
import torch
import torch.nn as nn
from loguru import logger
import warnings
warnings.simplefilter("ignore", UserWarning)

Let's use the mads_datasets package (see [github](https://github.com/raoulg/mads_datasets) for more details) which I created for these lessons to give everyone easy access to the datasets we use for training.

In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor

for dataset in DatasetType:
    print(dataset)

DatasetType.FLOWERS
DatasetType.IMDB
DatasetType.GESTURES
DatasetType.FASHION
DatasetType.SUNSPOTS
DatasetType.IRIS
DatasetType.PENGUINS
DatasetType.FAVORITA
DatasetType.SECURE


There are a few datasets. For images, we can use FLOWERS (~3000 photos of flowers in 5 categories) and FASHION (60k fashion icons 28x28 pixels big).

Lets start with our good'ol MNIST.

In [3]:

fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
batchsize = 64
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2025-11-07 10:46:12.362 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\NvanOort\.cache\mads_datasets\fashionmnist
2025-11-07 10:46:12.362 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\NvanOort\.cache\mads_datasets\fashionmnist\fashionmnist.pt
c:\Users\NvanOort\Code\MADS\DLDEPL\portfolio-NTvanOort\.venv\Lib\site-packages\mads_datasets\factories\torchfactories.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded

We can obtain an item:

In [ ]:
x, y = next(iter(trainstreamer))
x.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

The image follows the channels-first convention: (channel, width, height). The label is an integer.

Let's re-use the model we had:

In [4]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

using cuda


In [5]:
from torch import nn
print(f"Using {device} device")

# Define model
class CNN(nn.Module):
    def __init__(self, filters, units1, units2, input_size=(32, 1, 28, 28)):
        super().__init__()
        self.in_channels = input_size[1]
        self.input_size = input_size
        self.filters = filters
        self.units1 = units1
        self.units2 = units2

        self.convolutions = nn.Sequential(
            nn.Conv2d(self.in_channels, filters, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
        )

        activation_map_size = self._conv_test(input_size)
        logger.info(f"Aggregating activationmap with size {activation_map_size}")
        self.agg = nn.AvgPool2d(activation_map_size)

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(filters, units1),
            nn.ReLU(),
            # nn.Dropout(0.4),
            nn.Linear(units1, units2),
            nn.ReLU(),
            # nn.Dropout(0.4),
            nn.Linear(units2, 10)
        )

    def _conv_test(self, input_size = (64, 1, 28, 28)):
        x = torch.ones(input_size)
        x = self.convolutions(x)
        return x.shape[-2:]

    def forward(self, x):
        x = self.convolutions(x)
        x = self.agg(x)
        logits = self.dense(x)
        return logits

model = CNN(filters=64, units1=64, units2=32).to(device)

2025-11-07 10:46:15.004 | INFO     | __main__:__init__:31 - Aggregating activationmap with size torch.Size([6, 6])


Using cuda:0 device


In [6]:
from mltrainer.imagemodels import CNNConfig, CNNblocks

In [ ]:
config = CNNConfig(
    matrixshape = (28, 28), # every image is 28x28
    batchsize = batchsize,
    input_channels = 1, # we have black and white images, so only one channel
    hidden = 32, # number of filters
    kernel_size = 3, # kernel size of the convolution
    maxpool = 3, # kernel size of the maxpool
    num_layers = 4, # we will stack 4 Convolutional blocks, each with two Conv2d layers
    num_classes = 10,
)

In [ ]:
model = CNNblocks(config)
model.config

Calculated matrix size: 9
Caluclated flatten size: 288


{'matrixshape': (28, 28),
 'batchsize': 64,
 'input_channels': 1,
 'hidden': 32,
 'kernel_size': 3,
 'maxpool': 3,
 'num_layers': 4,
 'num_classes': 10}

In [ ]:
from torchinfo import summary
summary(model, input_size=(32, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNNblocks                                [32, 10]                  --
├─ModuleList: 1-1                        --                        --
│    └─ConvBlock: 2-1                    [32, 32, 28, 28]          --
│    │    └─Sequential: 3-1              [32, 32, 28, 28]          9,568
│    └─ConvBlock: 2-2                    [32, 32, 28, 28]          --
│    │    └─Sequential: 3-2              [32, 32, 28, 28]          18,496
│    └─ReLU: 2-3                         [32, 32, 28, 28]          --
│    └─MaxPool2d: 2-4                    [32, 32, 9, 9]            --
│    └─ConvBlock: 2-5                    [32, 32, 9, 9]            --
│    │    └─Sequential: 3-3              [32, 32, 9, 9]            18,496
│    └─ReLU: 2-6                         [32, 32, 9, 9]            --
│    └─ConvBlock: 2-7                    [32, 32, 9, 9]            --
│    │    └─Sequential: 3-4              [32, 32, 9, 9]            18,496


And set up the optimizer, loss and accuracy.

In [7]:
import torch.optim as optim
from mltrainer import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [ ]:
yhat = model(x.to(device))
accuracy(y.to(device), yhat)

0.140625

In [ ]:
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="demo",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TOML],
)

In [ ]:
trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optimizer,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
trainer.loop()

2025-10-27 14:46:53.076 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to demo\20251027-144653
2025-10-27 14:46:56.288 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:01<00:00, 60.43it/s]
2025-10-27 14:46:58.540 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.8107 test 0.9269 metric ['0.6595']
100%|██████████| 100/100 [00:01<00:00, 77.89it/s]
2025-10-27 14:47:00.399 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.8346 test 0.7319 metric ['0.6945']
100%|██████████| 100/100 [00:01<00:00, 77.33it/s]
2025-10-27 14:47:02.267 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.7231 test 0.7100 metric ['0.7305']
100%|██████████| 3/3 [00:05<00:00,  1.99s/it]


# MLflow
MLflow is an open-source platform designed to manage the entire Machine Learning (ML) lifecycle, including experimentation, reproducibility, deployment, and governance. It provides a set of APIs and tools to streamline ML workflows, making it easier to track experiments, package code, manage model versions, and deploy models.

Reasons to use MLflow over TensorBoard, gin-config, or Ray:

- End-to-end ML lifecycle management: While TensorBoard focuses on visualizing model training metrics and gin-config on hyperparameter configuration, MLflow covers a broader range of tasks, such as experiment tracking, model packaging, and deployment.

- Framework agnostic: MLflow is not tied to a specific ML framework, making it suitable for projects using different libraries or even multiple libraries.

- Model Registry: MLflow provides a centralized model registry, allowing you to version, track, and manage your models, which is not available in TensorBoard or gin-config.

- Deployment support: MLflow facilitates model deployment to various platforms, such as local, cloud, or Kubernetes environments, whereas TensorBoard and gin-config are not built for deployment tasks.

- Integration with other tools: MLflow integrates with popular tools and platforms like Databricks, AWS, and Azure, making it easy to incorporate into existing workflows.

However, the choice between MLflow and other tools like TensorBoard, gin-config, or Ray depends on your specific use case and the scope of the ML workflow you want to manage.

In [8]:
experiment_path = "mlflow_test"

In [9]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_path)

2025/11/07 10:46:38 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/11/07 10:46:38 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='file:c:/Users/NvanOort/Code/MADS/DLDEPL/portfolio-NTvanOort/2-hypertuning-mlflow/mlruns/1', creation_time=1760812635414, experiment_id='1', last_update_time=1760812635414, lifecycle_stage='active', name='mlflow_test', tags={}>

In the code above, we set the MLflow tracking URI to a local SQLite database file. This is done to configure the storage location for MLflow's experiment tracking data, such as metrics, parameters, and artifacts. By specifying a SQLite database, we enable a lightweight and easy-to-use storage solution for tracking the experiments and their associated information.

The line mlflow.set_experiment("mnist_convolutions") sets the active MLflow experiment to "mnist_convolutions". This is useful for organizing and grouping your runs, as it allows you to associate the upcoming ML training runs with a specific experiment name, making it easier to search, compare, and analyze the results later.

In [10]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

We import functions and classes from the hyperopt library to perform hyperparameter optimization. This library helps us find the best hyperparameter values for our machine learning model by searching through a defined search space and using optimization algorithms like Tree-structured Parzen Estimator (TPE). The goal is to improve our model's performance by tuning its hyperparameters.

Advantages of TPE:

- Model-based approach: TPE is a Bayesian optimization method that models the objective function as a probability distribution. It learns from previous evaluations to decide which points in the search space to explore next, making it more efficient in finding optimal hyperparameters.

- Exploration-exploitation trade-off: TPE balances the trade-off between exploration (searching in new regions of the search space) and exploitation (refining around the current best points). This can lead to better results in problems with complex search spaces.

- Continuous hyperparameter optimization: TPE can handle continuous hyperparameters more naturally, as it builds a probability model to estimate the performance for any given point in the search space.

Lets set up an objective function and start logging some usefull things we might want to track:

In [11]:
modeldir = Path("models").resolve()
if not modeldir.exists():
    modeldir.mkdir()
    print(f"Created {modeldir}")

In [18]:
import torch.optim as optim
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
from datetime import datetime

# Define the hyperparameter search space
settings = TrainerSettings(
    epochs=20,
    metrics=[accuracy],
    logdir=modeldir,
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.MLFLOW, ReportTypes.TOML],
    optimizer_kwargs={"lr": 0.0015, "weight_decay": 1e-4},
)


# Define the objective function for hyperparameter optimization
def objective(params):
    # Start a new MLflow run for tracking the experiment
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", "convnet")
        mlflow.set_tag("dev", "nick")
        # Log hyperparameters to MLflow
        mlflow.log_params(params)
        mlflow.log_param("batchsize", f"{batchsize}")


        # Initialize the optimizer, loss function, and accuracy metric
        optimizer = optim.AdamW
        loss_fn = torch.nn.CrossEntropyLoss()
        accuracy = metrics.Accuracy()
        config = CNNConfig(
            matrixshape = (28, 28), # every image is 28x28
            batchsize = batchsize,
            input_channels = 1, # we have black and white images, so only one channel
            hidden = params["filters"], # number of filters
            kernel_size = params["kernel_size"], # kernel size of the convolution
            maxpool = params["maxpool"], # kernel size of the maxpool
            num_layers = params["num_layers"], # we will stack 4 Convolutional blocks, each with two Conv2d layers
            num_classes = 10,
        )

        # Instantiate the CNN model with the given hyperparameters
        model = CNNblocks(config)
        # Train the model using a custom train loop
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optimizer,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        # Save the trained model with a timestamp
        tag = datetime.now().strftime("%Y%m%d-%H%M")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

        # Log the saved model as an artifact in MLflow
        mlflow.log_artifact(local_path=modelpath, artifact_path="pytorch_models")
        return {'loss' : trainer.test_loss, 'status': STATUS_OK}

In [19]:
# Definieer handmatig je parameters
params = {
    'filters': 64,
    'kernel_size': 3,
    'maxpool': 2,
    'num_layers': 2
}

# Roep de objective functie aan
result = objective(params)
print(f"Test loss: {result['loss']}")

2025-11-07 11:03:26.536 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to C:\Users\NvanOort\Code\MADS\DLDEPL\portfolio-NTvanOort\2-hypertuning-mlflow\models\20251107-110326


Calculated matrix size: 196
Caluclated flatten size: 12544


2025-11-07 11:03:27.296 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:02<00:00, 39.67it/s]
2025-11-07 11:03:30.767 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.9435 test 0.6426 metric ['0.7628']
100%|██████████| 100/100 [00:02<00:00, 43.18it/s]
2025-11-07 11:03:34.074 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.5239 test 0.5341 metric ['0.8031']
100%|██████████| 100/100 [00:02<00:00, 43.10it/s]
2025-11-07 11:03:37.398 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.4503 test 0.4498 metric ['0.8394']
100%|██████████| 100/100 [00:02<00:00, 44.26it/s]
2025-11-07 11:03:40.598 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.3993 test 0.3686 metric ['0.8641']
100%|██████████| 100/100 [00:02<00:00, 43.69it/s]
2025-11-07 11:03:43.872 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.3763 test 0.3623 metric ['0.8636']
100%|██

Test loss: 0.2606255742907524


See https://hyperopt.github.io/hyperopt/getting-started/search_spaces/ for more information about searchspaces for hyperopt

In [ ]:
search_space = {
    'filters' : scope.int(hp.quniform('filters', 16, 128, 8)),
    'kernel_size' : scope.int(hp.quniform('kernel_size', 2, 5, 1)),
    'num_layers' : scope.int(hp.quniform('num_layers', 1, 10, 1)),
}

We define a search space for hyperparameter optimization using Hyperopt. The search space specifies the range and distribution of hyperparameters to explore during the optimization process. This is crucial for finding the optimal set of hyperparameters that yield the best performance for the machine learning model. The search space defined here includes the number of filters in the convolutional layers, and the number of units in two fully connected layers, allowing Hyperopt to find the best combination within the given ranges.


Now, finally, let us perform the hyperparameter search using the fmin function from hyperopt. The function takes the following arguments:

- `fn=objective`: The objective function to minimize, which is defined earlier to train the model and return the test loss.
- `space=search_space`: The search space defined earlier, containing the range of hyperparameters to explore.
- `algo=tpe.suggest`: The optimization algorithm to use, in this case, the Tree-structured Parzen Estimator (TPE) method.
- `max_evals=10`: The maximum number of function evaluations, i.e., the maximum number of hyperparameter combinations to try.
- `trials=Trials()`: A Trials object to store the results of each evaluation.

The fmin function searches for the best hyperparameters within the given search space using the TPE algorithm, aiming to minimize the objective function (test loss). Once the optimization process is completed, the best hyperparameters found are stored in the best_result variable.

In [28]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)

Calculated matrix size: 49                           
Caluclated flatten size: 3920                        
  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

2025-11-07 11:15:17.589 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to C:\Users\NvanOort\Code\MADS\DLDEPL\portfolio-NTvanOort\2-hypertuning-mlflow\models\20251107-111517
2025-11-07 11:15:18.373 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
  0%|          | 0/20 [00:00<?, ?it/s]
ERROR [hyperopt.fmin] job exception: mat1 and mat2 shapes cannot be multiplied (64x80 and 3920x80)


  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x80 and 3920x80)

After running this, you can look at the best_result

In [ ]:
best_result

# MLflow GUI
MLflow has a really great dashboard.
you can see it with the command:
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
```

The `--backend-store-uri` argument specifies the location of the SQLite database file, which is used to store experiment metadata, such as parameters, metrics, and artifacts. 
We have initialized the experiment with `mlflow.set_tracking_uri("sqlite:///mlflow.db")`, so `mlflow.db` is the location we need. 

`--host` tells the server we use our own machine (localhost), and `--port` specifies the port number on which the server will listen for incoming requests. In this case, we are using port 5000. Sometimes, you can have conflicts on a specific port and it could help to change the port (eg to 5001)

Note that on a windows machine, the `\` gives errors (because, why not, right) so for windows you might need to remove the `\` and put everything on a single line.

I have created a `Makefile` to automate these commands, but if it doesnt work (again, because you are on windows for example) you can just type the command by hand in your terminal.

After starting this up, go to `http://127.0.0.1:5000` in your browser. You should see the MLflow UI, where you can explore your experiments, runs, and metrics. The UI provides a user-friendly way to visualize and compare different runs, making it easier to analyze the results of your hyperparameter optimization and model training.